In [ ]:
import pandas as pd
import json
import numpy as np

# Загружаем данные из JSON файла
with open('/kaggle/input/web-robot-detection/public_v2.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Преобразуем данные в список словарей, подготовив их для pandas DataFrame
data_list = [dict(id=key, **value) for key, value in data.items()]

# Создаем DataFrame
test_file = pd.DataFrame(data_list)

# Показываем первые строки DataFrame для проверки 
print(test_file.head())


In [ ]:
# Список колонок, которые нужно оставить в DataFrame
need_col = ['request', 'response', 'ip', 'useragent', 'timestamp']

# Создание нового DataFrame с выбранными колонками из исходного DataFrame
df = test_file[need_col]

# Удаление строк с отсутствующими значениями (NaN) в любой из колонок
df = df.dropna(axis=0)


In [ ]:
# Преобразование столбца 'timestamp' в формат datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Извлечение нужной информации из столбца 'request' и замена значений
df['request'] = df['request'].apply(lambda x: str(x).split()[7].split('/')[1] if (str(x).split()[7].split('/')[0]=="") else "Timeout")

# Замена значений в столбце 'request' на 'rc4.js?' если начало строки 'rc4.js?'
df['request'] = df['request'].apply(lambda x: "rc4.js?" if x[:7]=="rc4.js?" else x)

# Замена значений в столбце 'request' на 'favicon.ico?' если начало строки 'favicon.ico?'
df['request'] = df['request'].apply(lambda x: "favicon.ico?" if x[:12]=="favicon.ico?" else x)

# Замена значений в столбце 'request' на 'sitemap-n.xml' если начало строки 'sitemap-'
df['request'] = df['request'].apply(lambda x: "sitemap-n.xml" if x[:8]=="sitemap-" else x)


In [ ]:
# Импорт LabelEncoder из библиотеки sklearn.preprocessing
from sklearn.preprocessing import LabelEncoder

# Создание экземпляра LabelEncoder
LE = LabelEncoder()

# Применение LabelEncoder к столбцу 'request' для преобразования категориальных значений в числовые
df['request'] = LE.fit_transform(df['request'])

# Сортировка DataFrame по столбцу 'timestamp'
df = df.sort_values(by=['timestamp'])

# Добавление столбца 'label', который указывает на принадлежность useragent к боту (1) или нет (0)
df['label'] = df['useragent'].apply(lambda x: 1 if 'bot' in str(x) else (1 if 'crawl' in str(x) else (1 if 'BUbiNG' in str(x) else (1 if 'Bot' in str(x) else (1 if 'Crawl' in str(x) else 0)))))

# Определение количества уникальных классов в столбце 'request'
num_classes = len(df['request'].unique())

# Сброс индексов DataFrame
df.reset_index(drop=True, inplace=True)


In [ ]:
# Вывод первых строк DataFrame для проверки результатов
df.head()


In [ ]:
# Определение уникальных значений в столбце 'ip'
log_entry = "ip"
unique_key = df[log_entry].unique()

# Определение количества уникальных значений в столбце 'ip'
unique_n = len(df[log_entry].unique())

# Создание словаря для хранения параметров для ботов
param_bot = dict()

# Создание словаря для хранения параметров для обычных пользователей
param_man = dict()

# Создание словаря для хранения логов для ботов
log_key_bot = dict()

# Создание словаря для хранения логов для обычных пользователей
log_key_man = dict()

# Инициализация пустых списков для каждого уникального значения ip в словарях
for i in range(unique_n):
    param_bot[unique_key[i]] = []
    param_man[unique_key[i]] = []
    log_key_bot[unique_key[i]] = []
    log_key_man[unique_key[i]] = []


In [ ]:
# Итерация по строкам DataFrame
for idx in range(len(df)):
    # Если метка равна 1 (бот)
    if df.iloc[idx]['label'] == 1:
        # Добавление временного интервала между текущим и предыдущим запросами и метки в словарь для ботов
        param_bot[df[log_entry][idx]].append((df.iloc[idx]['timestamp'] - df.iloc[idx - 1]['timestamp'], df.iloc[idx]["label"]))
        # Добавление кортежа (запрос, метка) в словарь логов для ботов
        log_key_bot[df[log_entry][idx]].append((df.iloc[idx]['request'], df.iloc[idx]["label"]))
    # Если метка равна 0 (обычный пользователь)
    elif df.iloc[idx]['label'] == 0:
        # Добавление временного интервала между текущим и предыдущим запросами и метки в словарь для обычных пользователей
        param_man[df[log_entry][idx]].append((df.iloc[idx]['timestamp'] - df.iloc[idx - 1]['timestamp'], df.iloc[idx]["label"]))
        # Добавление кортежа (запрос, метка) в словарь логов для обычных пользователей
        log_key_man[df[log_entry][idx]].append((df.iloc[idx]['request'], df.iloc[idx]["label"]))


In [ ]:
# Инициализация списков для последовательностей параметров и логов для ботов и обычных пользователей
seq_param_bot = []
seq_param_man = []
seq_log_key_bot = []
seq_log_key_man = []

# Добавление значений параметров и логов для ботов в соответствующие списки
for k in param_bot.keys():
    seq_param_bot.append(param_bot[k])

# Добавление значений параметров и логов для обычных пользователей в соответствующие списки
for k in param_man.keys():
    seq_param_man.append(param_man[k])

# Добавление значений логов запросов для ботов в соответствующий список
for k in log_key_bot.keys():
    seq_log_key_bot.append(log_key_bot[k])

# Добавление значений логов запросов для обычных пользователей в соответствующий список
for k in log_key_man.keys():
    seq_log_key_man.append(log_key_man[k])

# Сортировка списков по длине последовательностей в обратном порядке
seq_param_bot.sort(key=len, reverse=True)
seq_param_man.sort(key=len, reverse=True)
seq_log_key_bot.sort(key=len, reverse=True)
seq_log_key_man.sort(key=len, reverse=True)


In [ ]:
# Нахождение индекса, до которого нужно оставить последовательности параметров для ботов
idx = len(seq_param_bot)
for item in range(len(seq_param_bot)):
    if len(seq_param_bot[item]) <= 5:
        idx = item
        break

# Обрезка списка последовательностей параметров для ботов
seq_param_bot = seq_param_bot[:idx]

# Нахождение индекса, до которого нужно оставить последовательности параметров для обычных пользователей
idx = len(seq_param_man)
for item in range(len(seq_param_man)):
    if len(seq_param_man[item]) <= 5:
        break

# Обрезка списка последовательностей параметров для обычных пользователей
seq_param_man = seq_param_man[:idx]

# Нахождение индекса, до которого нужно оставить последовательности логов запросов для ботов
idx = len(seq_log_key_bot)
for item in range(len(seq_log_key_bot)):
    if len(seq_log_key_bot[item]) <= 5:
        idx = item
        break

# Обрезка списка последовательностей логов запросов для ботов
seq_log_key_bot = seq_log_key_bot[:idx]

# Нахождение индекса, до которого нужно оставить последовательности логов запросов для обычных пользователей
idx = len(seq_log_key_man)
for item in range(len(seq_log_key_man)):
    if len(seq_log_key_man[item]) <= 5:
        idx = item
        break

# Обрезка списка последовательностей логов запросов для обычных пользователей
seq_log_key_man = seq_log_key_man[:idx]


In [ ]:
# Вывод первых 10 элементов первой последовательности параметров для обычных пользователей
seq_param_man[0][:10]


In [ ]:
# Вывод первых 10 элементов шестой последовательности логов запросов для ботов
seq_log_key_bot[5][:10]


In [ ]:
# Инициализация переменной для подсчета общего числа элементов во всех последовательностях логов запросов для ботов
nums = 0

# Вычисление общего числа элементов во всех последовательностях логов запросов для ботов
for seq in seq_log_key_bot:
    nums += len(seq)

# Определение коэффициента разделения на обучающую и валидационную выборки
ratio = 0.8

# Вычисление числа элементов, которые будут использованы для обучения
train_num = int(nums * ratio)

# Инициализация переменной для хранения временной суммы элементов
tmp = 0

# Инициализация списка для хранения ключей обучающей выборки
key_train = []

# Инициализация списка для хранения ключей валидационной выборки
key_valid = []

# Разделение каждой последовательности логов запросов для ботов на обучающую и валидационную части
for seq in seq_log_key_bot:
    # Вычисление индекса, до которого идут элементы обучающей выборки
    tmp = len(seq)
    idx = int(ratio * tmp)
    # Добавление элементов до этого индекса в обучающую выборку
    key_train.append(seq[:idx])
    # Добавление элементов после этого индекса в валидационную выборку
    key_valid.append(seq[idx:])


In [ ]:
# Определение коэффициента разделения на обучающую и валидационную выборки
ratio = 0.8

# Вычисление числа элементов, которые будут использованы для обучения
train_num = int(nums * ratio)

# Инициализация переменной для хранения временной суммы элементов
tmp = 0

# Инициализация списка для хранения параметров обучающей выборки
param_train = []

# Инициализация списка для хранения параметров валидационной выборки
param_valid = []

# Разделение каждой последовательности параметров для ботов на обучающую и валидационную части
for seq in seq_param_bot:
    # Вычисление индекса, до которого идут элементы обучающей выборки
    tmp = len(seq)
    idx = int(ratio * tmp)
    # Добавление элементов до этого индекса в обучающую выборку
    param_train.append(seq[:idx])
    # Добавление элементов после этого индекса в валидационную выборку
    param_valid.append(seq[idx:])


In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
import time
import tensorflow as tf
import os

# Функция для генерации входных и выходных данных для модели
def generate(name, window_size):
    num_sessions = 0
    inputs = []
    outputs = []

    # Проход по каждой сессии в наборе данных
    for line in name:
        num_sessions += 1
        # Проход по каждому элементу в сессии, кроме последних window_size элементов
        for i in range(len(line) - window_size):
            inputs_tmp = []
            # Формирование входных данных как последовательности из window_size элементов
            for j in line[i:i + window_size]:
                inputs_tmp.append(j[0])
            inputs.append(inputs_tmp)
            # Формирование выходных данных как следующего элемента после входной последовательности
            outputs.append(line[i + window_size][0])
    return inputs, outputs

# Размер окна для последовательности
window_size = 10

# Количество классов
num_classes = len(df['request'].unique())

# Инициализация переменных для оценки производительности модели
TP = 0
FP = 0

# Количество кандидатов для предсказания
n_candidates = 10


In [ ]:
# Генерация обучающих данных
X, Y = generate(key_train, window_size)
# Преобразование формы входных данных
X = np.reshape(X, (len(X), window_size, 1))
# Преобразование выходных данных в one-hot encoding
Y = to_categorical(Y, num_classes)

# Генерация валидационных данных
X_valid, Y_valid = generate(key_valid, window_size)
# Преобразование формы входных данных для валидации
X_valid = np.reshape(X_valid, (len(X_valid), window_size, 1))
# Преобразование выходных данных в one-hot encoding для валидации
Y_valid = to_categorical(Y_valid, num_classes)


In [ ]:
X[5][:5]

In [ ]:
Y[5][:5]

In [ ]:
# Функция для генерации входных и выходных данных для модели на основе параметров
def generate_param(name, window_size):
    num_sessions = 0
    inputs = []
    outputs = []

    # Проход по каждой сессии в наборе данных
    for line in name:
        num_sessions += 1
        # Проход по каждому элементу в сессии, кроме последних window_size элементов
        for i in range(len(line) - window_size):
            inputs_tmp = []
            # Формирование входных данных как последовательности из window_size элементов
            for j in line[i:i + window_size]:
                inputs_tmp.append(j[0])
            inputs.append(inputs_tmp)
            # Формирование выходных данных как следующего элемента после входной последовательности
            outputs.append(line[i + window_size][0])
    return inputs, outputs


In [ ]:
# Генерация обучающих данных на основе параметров
X_p, Y_p = generate_param(param_train, window_size)

# Генерация валидационных данных на основе параметров
X_p_valid, Y_p_valid = generate_param(param_valid, window_size)


In [ ]:
# Преобразование элементов входных данных обучающей выборки в секунды и деление на 10
for i in range(len(X_p)):
    for j in range(len(X_p[i])):
        X_p[i][j] = int(X_p[i][j].total_seconds()) // 10

# Преобразование элементов выходных данных обучающей выборки в секунды и деление на 10
for i in range(len(Y_p)):
    Y_p[i] = int(Y_p[i].total_seconds()) // 10

# Преобразование элементов входных данных валидационной выборки в секунды и деление на 10
for i in range(len(X_p_valid)):
    for j in range(len(X_p_valid[i])):
        X_p_valid[i][j] = int(X_p_valid[i][j].total_seconds()) // 10

# Преобразование элементов выходных данных валидационной выборки в секунды и деление на 10
for i in range(len(Y_p_valid)):
    Y_p_valid[i] = int(Y_p_valid[i].total_seconds()) // 10

# Создание нового списка для хранения уникальных URL
new_list = []

# Множество для хранения уникальных URL
url_set = set()

# Проход по каждому элементу входных данных обучающей выборки
for item in X_p:
    # Если URL отсутствует в множестве, добавляем его в новый список и множество
    if item[2] not in url_set:
        url_set.add(item[2])
        new_list.append(item[2])

# Проход по каждому элементу выходных данных обучающей выборки
for item in Y_p:
    # Если URL отсутствует в множестве, добавляем его в новый список и множество
    if item not in url_set:
        url_set.add(item)
        new_list.append(item)


In [ ]:
# Количество параметров
num_params = 30  # от 0 секунд до 300 секунд

# Преобразование входных данных обучающей выборки в массив numpy и изменение формы
X_p = np.array(X_p).reshape(-1, 10, 1)
# Преобразование выходных данных обучающей выборки в one-hot encoding
targets = np.array([Y_p]).reshape(-1)
Y_p = np.eye(num_params)[targets]

# Преобразование входных данных валидационной выборки в массив numpy и изменение формы
X_p_valid = np.array(X_p_valid).reshape(-1, 10, 1)
# Преобразование выходных данных валидационной выборки в one-hot encoding
targets = np.array([Y_p_valid]).reshape(-1)
Y_p_valid = np.eye(num_params)[targets]


In [ ]:
X_p[41981]

In [ ]:
Y_p[5]

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Размер выходного слоя
output_size = Y.shape[1]

# Размер пакета обучающих данных
batch_size = 2000

# Оптимизатор
optimizer = Adam(lr=3e-4)

# Количество эпох
epoch_num = 100

# Название файла для сохранения весов модели
filename = 'checkpoint-epoch-{}-trial02.h5'.format(epoch_num)

# Коллбэк для сохранения весов модели
checkpoint_callback = ModelCheckpoint(filename,             
                                       monitor='val_accuracy',   
                                       verbose=1,            
                                       save_best_only=True,  
                                       mode='auto'          
                                      )

# Коллбэк для ранней остановки
early_stopping = EarlyStopping(monitor='val_accuracy',  
                               patience=10,         
                              )

# Определение архитектуры модели
model = Sequential()
model.add(LSTM(512, activation='relu', return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256, return_sequences=False))
model.add(Dense(output_size, activation='softmax'))

# Компиляция модели
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

# Обучение модели
model.fit(X, Y, epochs=epoch_num, validation_data=(X_valid, Y_valid), 
          callbacks=[checkpoint_callback, early_stopping], batch_size=batch_size, shuffle=True)


In [ ]:
# Размер выходного слоя
output_size = 30

# Размер пакета обучающих данных
batch_size = 2000

# Оптимизатор
optimizer = Adam(lr=3e-4)

# Количество эпох
epoch_num = 100

# Название файла для сохранения весов модели
filename = 'checkpoint-epoch-{}-trial02.h5'.format(epoch_num)

# Коллбэк для сохранения весов модели
checkpoint_callback = ModelCheckpoint(filename,             
                                       monitor='val_accuracy',   
                                       verbose=1,            
                                       save_best_only=True,  
                                       mode='auto'          
                                      )

# Коллбэк для ранней остановки
early_stopping = EarlyStopping(monitor='val_accuracy',  
                               patience=10,         
                              )

# Определение архитектуры модели
model2 = Sequential()
model2.add(LSTM(128, activation='relu', return_sequences=True, input_shape=(X_p.shape[1], X_p.shape[2])))
model2.add(LSTM(64, return_sequences=True))
model2.add(LSTM(32, return_sequences=False))
model2.add(Dense(output_size, activation='softmax'))

# Компиляция модели
model2.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

# Обучение модели
model2.fit(X_p, Y_p, epochs=epoch_num, validation_data=(X_p_valid, Y_p_valid), 
           callbacks=[checkpoint_callback, early_stopping], batch_size=batch_size, shuffle=True)


In [ ]:
# Преобразование элементов последовательностей параметров для обычных пользователей в список
for i in range(len(seq_param_man)):
    for j in range(len(seq_param_man[i])):
        seq_param_man[i][j] = list(seq_param_man[i][j])

# Преобразование времени в секунды и деление на 10
for i in range(len(seq_param_man)):
    for j in range(len(seq_param_man[i])):
        seq_param_man[i][j][0] = int(seq_param_man[i][j][0].total_seconds()) // 10


In [ ]:
# Функция для генерации данных для предсказания
def generate_pred(file, window_size):
    hdfs = list()
    haaa = list()
    hhhh = list()

    uri = []
    time = []
    trid = []

    # Проход по каждой строке в файле
    for line in file:
        uri_tmp = []
        trid_tmp = []
        time_tmp = []
        # Извлечение URI, времени и идентификаторов транзакций из строки
        for i in line:
            uri_tmp.append(i[0])
            time_tmp.append(int(i[1].total_seconds()) // 10)
            trid_tmp.append(i[2])
        uri.append(uri_tmp)
        time.append(time_tmp)
        trid.append(trid_tmp)
    
    # Добавление заполнения последовательности, когда она короче, чем размер окна
    for ln in uri:
        line = list(map(lambda n: n - 1, ln))
        ln = line + [-2] * (window_size + 1 - len(line))
        hdfs.append(tuple(ln))

    for ll in trid:
        line = list(ll)
        ll = line + [-2] * (window_size + 1 - len(line))
        hhhh.append(tuple(ll))
        
    for l in time:
        line = list(l)
        l = line + [-2] * (window_size + 1 - len(line))
        haaa.append(tuple(l))

    return hdfs, haaa, hhhh


In [ ]:
# Создание словаря для хранения данных об обычных пользователях
man = dict()

# Инициализация пустых списков для каждого уникального ключа
for i in range(unique_n):
    man[unique_key[i]] = []

# Наполнение словаря данными
for idx in range(len(df)):
    # Если метка равна 0 (обычный пользователь)
    if df.iloc[idx]['label'] == 0:
        # Добавление данных в соответствующий список в словаре
        man[df[log_entry][idx]].append((df.iloc[idx]['request'], df.iloc[idx]['timestamp'] - df.iloc[idx - 1]['timestamp'], df.iloc[idx]["label"]))

# Создание списка для хранения данных обычных пользователей
man_ = []

# Преобразование словаря в список значений и сортировка по длине в обратном порядке
for k in man.keys():
    man_.append(man[k])
man_.sort(key=len, reverse=True)


In [ ]:
# Инициализация переменной для хранения индекса среза списка
idx = len(man_)

# Поиск индекса, до которого нужно сохранить элементы списка
for item in range(len(man_)):
    if len(man_[item]) <= 5:
        idx = item
        break

# Срез списка до найденного индекса
man_ = man_[:idx]


In [ ]:
test_key_normal_loader, test_normal_loader, y_test = generate_pred(man_, window_size)

In [ ]:
from tqdm import tqdm
import keras
from tensorflow.keras.activations import softmax

# Инициализация переменных для подсчета статистики
total = 0
correct = 0
fail = 0
human_count = 0
proba = []
y_labeled = []
start_time = time.time()

# Перебор данных из тестового набора с помощью tqdm для отображения прогресса
for line, line2, y in tqdm(zip(test_key_normal_loader, test_normal_loader, y_test)):
    compare_int = 0
    # Проход по каждому элементу в строке, кроме последних window_size элементов
    for i in range(len(line) - window_size):
        compare = int(len(line) * 0.3)
        seq = line[i:i + window_size]
        seq_param = line2[i:i + window_size]
        label = line[i + window_size]
        label_param = line2[i + window_size]
        trid = y[i + window_size]
        
        # Пропуск, если метка равна -2
        if label == -2:
            continue

        # Преобразование входных данных и выходных данных в соответствующий формат
        X = np.reshape(seq, (1, window_size, 1))
        X = X / float(num_classes)
        Y = to_categorical(label, num_classes)
        
        # Предсказание метки с помощью модели
        prediction = model.predict(X, verbose=0)
        predicted = prediction.argsort()[0][::-1][:n_candidates]
        y_pred = prediction

        proba.append(y_pred)
        total += 1

        # Если верная метка присутствует в n_candidates лучших предсказаниях
        if np.argmax(Y) in prediction.argsort()[0][::-1][:n_candidates]:
            # Преобразование входных данных параметров в соответствующий формат
            Xp = np.reshape(seq_param, (1, window_size, 1))
            Xp = Xp / float(30)
            Yp = to_categorical(label_param, 30)
            
            # Предсказание метки параметров с помощью модели2
            prediction2 = model2.predict(Xp, verbose=0)
            if np.argmax(Yp) in prediction2.argsort()[0][::-1]:
                correct += 1
            else:
                compare_int += 1
                if (compare_int >= compare):
                    human_count += 1
                    break
        else:
            compare_int += 1
            if (compare_int >= compare):
                human_count += 1
                break
            
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))
print("total : %d" % total)
accu = human_count / len(man_) * 100
print("accuracy : %f" % accu)
